## 'IT 기술', 'BT 기술', 'NT 기술', 'ET 기술', 'CT 기술', 'ST 기술'로 서치

언론사별 서치(임의로 KBS로 서치)

In [1]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, progressbar, json, glob, time
from tqdm import tqdm

In [2]:
###### 날짜 저장 ##########
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5

In [14]:
####### 언론사별 본문 위치 태그 파싱 함수 ###########
def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('div', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
            
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = soup.find('div', {'class' : 'art_txt'}).text
        
    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
    # KBS
    elif 'news.kbs' in url:
        try:
            text = soup.find('div', {'id' : 'cont_newstext'}).text
        except:
            return "x" 
        
    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 그 외
    else:
        text == None
        
    return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    
    
press_nm = 'KBS'

print('검색할 언론사 : {}'.format(press_nm))



본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : KBS


In [29]:
def _news_crawling(query, news_num): 
    ############### 브라우저를 켜고 검색 키워드 입력 ####################
    news_num = int(news_num)

    chrome_path = 'C:/chromedriver/chromedriver.exe'
    browser = webdriver.Chrome(chrome_path)

    news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(query)
    browser.get(news_url)
    time.sleep(sleep_sec)


    ######### 언론사 선택 및 confirm #####################
    search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
    search_opn_btn.click()
    time.sleep(sleep_sec)

    bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

    # 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
    press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
    press_tablist[1].click()
    time.sleep(sleep_sec)

    # 첫 번째 것(언론사 분류선택)
    bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

    press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
    press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


    for press_kind_btn in press_kind_btn_list:

        # 언론사 종류를 순차적으로 클릭(좌측)
        press_kind_btn.click()
        time.sleep(sleep_sec)

        # 언론사선택(우측)
        press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
        # 언론사 선택할 수 있는 클릭 버튼
        press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
        # 언론사 이름들 추출
        press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]

        # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
        press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))

        # 원하는 언론사가 해당 이름 안에 있는 경우
        # 1) 클릭하고
        # 2) 더이상 언론사분류선택 탐색 중지
        if press_nm in press_slct_btn_dict.keys():
            press_slct_btn_dict[press_nm].click()
            time.sleep(sleep_sec)

            break



    ################ 뉴스 크롤링 ########################
    # ####동적 제어로 페이지 넘어가며 크롤링
    news_dict = {}
    idx = 1
    cur_page = 1

    pbar = tqdm(total=news_num ,leave = True)

    while idx < news_num:

        table = browser.find_element_by_xpath('//ul[@class="list_news"]')
        li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
        area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
        a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]

        for n in a_list[:min(len(a_list), news_num-idx+1)]:
            n_url = n.get_attribute('href')
            news_dict[idx] = {'title' : n.get_attribute('title'), 
                              'url' : n_url,
                              'text' : crawling_main_text(n_url)}

            idx += 1
            pbar.update(1)

        if idx < news_num:
            cur_page +=1

            pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
            next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

            browser.get(next_page_url)
            time.sleep(sleep_sec)
        else:
            pbar.close()
            time.sleep(0.7)
            browser.close()
            break

    #### 데이터 전처리하기 ###################################################### 
    news_df = DataFrame(news_dict).T

    folder_path = os.getcwd()
    xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(news_num, query, date)

    news_df.to_excel(xlsx_file_name)

    os.startfile(folder_path)

    return news_df

In [30]:
_news_crawling('정보기술', 30)

C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\jessy\Anaconda3\envs\jipark\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_ele

,title,url,text
1,정부 “가상자산 불공정 거래 행위 강력 대응할것”,https://news.kbs.co.kr/news/view.do?ncd=535898...,정부가 가상자산사업자 신고제 시행 이후 시장 상황을 점검하며 앞으로 불공정 거래 행...
2,방송사 간 소유 제한 폐지…유료 방송 규제 완화,https://news.kbs.co.kr/news/view.do?ncd=535879...,과학기술정보통신부(과기정통부)는 유료방송 산업 규제를 완화하는 '방송법 시행령' 및...
3,"“생활제품·유아동시설 전자파, 인체보호기준 만족”",https://news.kbs.co.kr/news/view.do?ncd=535881...,과학기술정보통신부가 생활제품 6종을 비롯해 유아·아동시설 등을 대상으로 전자파를 측...
4,참여연대 “LTE 상용화 10년간 이통3사 18조 넘는 이익 거둬”,https://news.kbs.co.kr/news/view.do?ncd=535882...,4세대 이동통신인 LTE 서비스가 상용화된 이후 10년간 이동통신3사가 각종 비용을...
5,"“성층권에서 재난 감시” 과기정통부, 장기 체공 드론 내년부터 개발",https://news.kbs.co.kr/news/view.do?ncd=535895...,과학기술정보통신부는 성층권에 한 달 이상 떠 있을 수 있는 재난감시용 드론을 개발하...
6,내년 산업부 R&D 예산 5조원 첫 돌파…탄소중립·디지털 집중,https://news.kbs.co.kr/news/view.do?ncd=535777...,정부의 내년도 산업기술 연구개발(R&D;) 지원 예산이 사상 처음으로 5조원을 돌파...
7,우리나라 방송 프로그램 수출액 사상 처음 4억 달러 돌파,https://news.kbs.co.kr/news/view.do?ncd=535778...,우리나라 방송 프로그램 수출액이 처음으로 4억 달러를 돌파했습니다. 과학기술정보통신...
8,오미크론에 IT 공룡기업 속속 CES 불참 선언…MS도 공식화,https://news.kbs.co.kr/news/view.do?ncd=535711...,세계 최대 소프트웨어 업체 마이크로소프트(MS)가 전염병 대유행을 이유로 세계 최대...
9,상반기 수사기관 제출 통신자료 256만여 건…지난해 대비 12.4%↓,https://news.kbs.co.kr/news/view.do?ncd=535653...,"올해 상반기 통신업체가 수사기관에 제출한 통신자료는 255만 9,439건으로 지난해..."
10,과기정통부 “개방형으로 사이버 위협정보 공유시스템 개편”,https://news.kbs.co.kr/news/view.do?ncd=535564...,"과학기술정보통신부는 오늘(23일), 더 많은 기업이 사이버 위협정보를 실시간으로 공..."


In [31]:
_news_crawling('생명공학기술', 30)

C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\jessy\Anaconda3\envs\jipark\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_ele

,title,url,text
1,"美, 위구르탄압 관련 中기관 제재",https://news.kbs.co.kr/news/view.do?ncd=535058...,미국 정부가 베이징 올림픽에 대해 외교적 보이콧을 선언한 데 이어 중국 정부기관과 ...
2,"경북도-서울대, 첨단생명공학 연구시설 준공",http://news.kbs.co.kr/news/view.do?ncd=5168573...,경상북도와 서울대는 경북 축산기술연구소 안에 첨단 생명공학 연구시설을 준공했습니다....
3,미래 감염병 ‘상시 R&D·전임상’으로 대응!,https://news.kbs.co.kr/news/view.do?ncd=535268...,[앵커] 코로나19 재확산을 계기로 그동안 한시적으로 운영됐던 감염병 연구협력체계...
4,美특허 출원 1위 대학은 카이스트…생명공학 가장 많아,http://news.kbs.co.kr/news/view.do?ncd=4210023...,국내 대학 가운데 미국에서 특허를 가장 많이 출원한 곳은 한국과학기술원(KAIST)...
5,“화학색소 없이 ‘구조색’ 컬러 인쇄 가능”,https://news.kbs.co.kr/news/view.do?ncd=535216...,[앵커] 공작새의 알록달록한 깃털 무늬나 나전칠기의 영롱한 빛깔은 흔히 자연에서만...
6,“화학 색소 없이 ‘구조색’ 컬러 인쇄 가능”,https://news.kbs.co.kr/news/view.do?ncd=535193...,[앵커] 공작새의 알록달록한 깃털 무늬나 나전칠기의 영롱한 빛깔은 흔히 자연에서만...
7,일반 세포를 ‘암 잡아내는’ 세포로 직접 전환…세계 최초 기술 개발,https://news.kbs.co.kr/news/view.do?ncd=525300...,일반 세포를 NK세포로 직접 바꾸는 기술을 국내 연구진이 세계 최초로 개발했습니다....
8,"美 “北, 일부 행태 개선 안 돼 테러지원국 잔류”…中 제재도 확대",https://news.kbs.co.kr/news/view.do?ncd=535101...,[앵커] 미국이 테러 관련 보고서에서 북한에 대해 여전히 문제 있고 우려스런 ...
9,항암 치료용 ‘NK세포 분화기술’ 상업화 기반 마련,http://news.kbs.co.kr/news/view.do?ncd=5154162...,"[앵커] 'NK 세포' 우리말로 자연살해세포인데, 암세포를 공격해 죽이기 때문에 ..."
10,"강원도, 농식품부 ‘그린바이오 벤처 캠퍼스’ 유치 도전",https://news.kbs.co.kr/news/view.do?ncd=522597...,강원도와 평창군이 농림축산식품부가 추진하는 ‘그린바이오 벤처 캠퍼스’ 유치에 도전합...


In [32]:
_news_crawling('나노기술', 30)

C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\jessy\Anaconda3\envs\jipark\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_ele

,title,url,text
1,"한국해양대, 탄소나노튜브 기술이전 협약",https://news.kbs.co.kr/news/view.do?ncd=532400...,한국해양대학교는 오늘 오전 프리원 주식회사와 촉매정제 공정이 필요 없는 탄소나노튜브...
2,"전기연구원 금속 잉크, ‘10대 나노기술’ 선정",http://news.kbs.co.kr/news/view.do?ncd=5062668...,한국전기연구원이 개발한 '저가형 금속·그래핀 복합잉크 제조기술'이 과학기술정보통신부...
3,"밀양 나노산단, 정작 나노업체 몰리지 않는 이유는?",https://news.kbs.co.kr/news/view.do?ncd=528405...,[앵커] 경남에서 추진되고 있는 국가산단의 현주소를 연속으로 짚어봅니다. '첨단 ...
4,"밀양 나노산단, 핵심 나노기업 유치가 관건",https://news.kbs.co.kr/news/view.do?ncd=527145...,[앵커]오는 2023년 준공 예정인 밀양 나노국가산업단지에 한국전력과 한국동서발전이...
5,"밀양 나노산단, 핵심 나노기업 유치가 관건",https://news.kbs.co.kr/news/view.do?ncd=527115...,[앵커] 오는 2023년 준공 예정인 밀양 나노국가산업단지에 한국전력과 한국동서발...
6,“커피 찌꺼기가 탄소 나노입자로”,https://news.kbs.co.kr/news/view.do?ncd=525606...,[앵커]유니스트의 벤처기업이 버려지는 커피 찌꺼기를 활용가치가 높은 탄소 나노입자...
7,올해 나노기술에 6천693억원 투자…산업화 촉진 나서,http://news.kbs.co.kr/news/view.do?ncd=3638902...,정부가 올해 나노기술 분야에 지난해보다 210억 원 늘어난 6천693억 원을 투자한...
8,[주진우 라이브] 이재갑 교수 “코로나 백신에 나노칩 넣어 인체 조종? 나노 로봇 ...,http://news.kbs.co.kr/news/view.do?ncd=5115885...,==============================================...
9,오래된 석조물 복원에 ‘나노 기술’ 활용,http://news.kbs.co.kr/news/view.do?ncd=3522483...,<리포트> 11세기 초 기공된 이탈리아 피사 대성당의 복원 공사가 한창입니다. ...
10,미래부 ‘4기 나노기술 발전계획안’ 공청회,http://news.kbs.co.kr/news/view.do?ncd=3183370...,미래창조과학부는 내일 서울 양재동 더케이 호텔에서 10개 부처 합동으로 나노 기...


In [33]:
_news_crawling('환경공학기술', 30)

C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\jessy\Anaconda3\envs\jipark\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_ele

,title,url,text
1,타이완과 ‘공급망·中 압박’ 논의한 美 국무차관…한국에서는?,https://news.kbs.co.kr/news/view.do?ncd=535063...,호세 W. 페르난데스(Jose W. Fernandez) 미국 국무부 경제성장·에너지...
2,일상회복위 자치안전 분과 2차 회의 개최…‘위드 코로나’ 의견 수렴,https://news.kbs.co.kr/news/view.do?ncd=530510...,행정안전부가 오늘(20일) 오후 3시 정부세종청사에서 코로나19 일상회복 지원위원회...
3,여성 과학기술 인재도 ‘국가인재DB’ 등록…주요직위에 활용,https://news.kbs.co.kr/news/view.do?ncd=518057...,"자연과학, 정보기술(IT), 에너지 분야 등의 여성과학기술 인재들이 ‘국가인재’로 ..."
4,성남 백현동 ‘50m 옹벽아파트’…수상한 안전성 심의?,https://news.kbs.co.kr/news/view.do?ncd=533838...,50m 높이의 수직 옹벽이 지어진 경기 성남시 백현동 판교 A아파트의 건축 심의 과...
5,[오태훈의 시사본부] 폭염센터장 “폭염은 법정 재난…태풍 오기 전까지 열돔 지속될 ...,https://news.kbs.co.kr/news/view.do?ncd=523415...,- 한반도 폭염 여러 발생 원인 있어 다양하고 복잡... 1~2주 전 정확한 예측 ...
6,[‘삼풍 참사’ 26주기 인터뷰]② “모든 재난 완벽히 막는 건 불가능…소 잃고 외...,https://news.kbs.co.kr/news/view.do?ncd=522049...,"[다시보기] 7월부터 새 거리두기, 달라지는 점은?/삼풍백화점 붕괴 참사 26주기,..."
7,"건조·강풍·때이른 이상고온, 대형 산불 키웠다",http://news.kbs.co.kr/news/view.do?ncd=5123835...,[앵커]2월에 대형 산불이 잇따르는 건 매우 이례적입니다.어제(21일) 하루에만 ...
8,한반도 태풍→美서부 산불 “영향 줬다”,http://news.kbs.co.kr/news/view.do?ncd=5087630...,"지난해 여름, 8월 22일 태풍 '바비'를 시작으로, '마이삭', '하이선' 등 9..."
9,"환경부, 그린뉴딜 이끌 녹색융합기술 특성화대학원 11개 선정",http://news.kbs.co.kr/news/view.do?ncd=5056393...,환경부와 한국환경산업기술원이 한국형 그린뉴딜을 선도할 '녹색 융합기술 특성화대학원'...
10,"동해안 근접해 태풍 북상, 왜?",http://news.kbs.co.kr/news/view.do?ncd=4533455...,[앵커] 이번에는 기상 전문가와 함께 이번 태풍 '하이선'의 영향에 대해 상세하게...


In [34]:
_news_crawling('문화콘텐츠기술', 30)

C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\jessy\Anaconda3\envs\jipark\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_ele

,title,url,text
1,기술보증기금 대전 문화콘텐츠금융센터 개점,http://news.kbs.co.kr/news/view.do?ncd=5120230...,기술보증기금 대전 문화콘텐츠 금융센터 개점식이 오늘(17일) 서구 둔산동에서 열렸습...
2,기술보증기금 대전 문화콘텐츠금융센터 개점,http://news.kbs.co.kr/news/view.do?ncd=5120720...,
3,"박물관·미술관 104곳, 실감콘텐츠 등 색다른 문화체험 제공",http://news.kbs.co.kr/news/view.do?ncd=5112125...,문화체육관광부가 '2021년 지능형(스마트) 박물관·미술관 기반 조성 사업' 지원 ...
4,“한국 전통문화 감상하며 입국하세요”…인천공항 미디어 아트 설치,http://news.kbs.co.kr/news/view.do?ncd=5149583...,국립고궁박물관은 인천국제공항 제1터미널 동편 입국장에 전통문화를 주제로 한 미디어아...
5,내년 3월부터 광화문서 ‘실감콘텐츠’ 체험…“광화시대 프로젝트 개막”,http://news.kbs.co.kr/news/view.do?ncd=5065162...,문화체육관광부와 한국콘텐츠진흥원은 내년 3월부터 문화·관광 콘텐츠와 5G 실감 기술...
6,문 대통령 “세계 선도하는 디지털콘텐츠 국가 도약할 것”,http://news.kbs.co.kr/news/view.do?ncd=5012185...,문재인 대통령은 디지털뉴딜 문화콘텐츠산업 전략 보고회를 주재하고 “한국판 뉴딜로 디...
7,"문 대통령, 문화혁신포럼 참석…“한국, 아세안 문화 콘텐츠 동반자 될 것”",http://news.kbs.co.kr/news/view.do?ncd=4330496...,"한-아세안 문화혁신포럼에 참석한 문재인 대통령은 ""한국이 같은 문화적 정체성 위에서..."
8,“추석 연휴 비대면 활동 지원…영상통화 무료·온라인 문화행사”,https://news.kbs.co.kr/news/view.do?ncd=528279...,중앙재난안전대책본부는 추석 연휴 기간 동안 코로나19 확산을 막기 위한 비대면 활동...
9,‘국내 유일’ 문화재산업전 9~11일 경주에서 개최,https://news.kbs.co.kr/news/view.do?ncd=527502...,국내 유일의 문화재산업전인 ‘2021 국제문화재산업전’이 오는 9일부터 11일까지 ...
10,"문체부, 공예·디자인상품 등 ‘2021 우수문화상품’ 57점 지정",https://news.kbs.co.kr/news/view.do?ncd=526742...,"문화체육관광부가 ‘2021 우수문화상품’으로 공예와 디자인상품, 한복, 문화콘텐츠,..."


In [35]:
_news_crawling('우주항공기술', 30)

C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/4202372388.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
C:\Users\jessy\Anaconda3\envs\jipark\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_ele

,title,url,text
1,이재명 “과학 부총리 도입…과학기술은 국가 생존의 문제”,https://news.kbs.co.kr/news/view.do?ncd=535426...,"더불어민주당 이재명 대선 후보가 과학기술혁신 부총리를 신설하고, 2030년까지 대한..."
2,블룸버그 “한국 누리호 궤도에 진입 할 것…우주항공 산업에 큰 발전”,https://news.kbs.co.kr/news/view.do?ncd=530594...,오늘 한국이 자체 개발한 우주 발사체인 누리호가 성공적으로 발사돼 궤도에 진입 할 ...
3,"정부, AI·첨단바이오 등 10개 분야 전략기술 지정",https://news.kbs.co.kr/news/view.do?ncd=535522...,"정부가 인공지능과 첨단바이오, 2차전지, 우주항공 등 10개 분야 기술을 국가 전략..."
4,"정부, AI·첨단바이오 등 10개 분야 전략기술 지정",https://news.kbs.co.kr/news/view.do?ncd=535493...,"정부가 인공지능과 첨단바이오, 2차전지, 우주항공 등 10개 분야 기술을 국가 전략..."
5,"경남서부, ‘항공우주 인증도시’ 도약",https://news.kbs.co.kr/news/view.do?ncd=532134...,[앵커] 비행기 사고에 대비해 부품이나 기체가 얼마나 안전한지 시험하는 과정이 매...
6,"[여기는 진주] 경남서부, ‘항공우주 인증도시’ 도약",https://news.kbs.co.kr/news/view.do?ncd=532093...,[앵커] 비행기 사고에 대비해 부품이나 기체가 얼마나 안전한지 시험하는 과정이 매...
7,"한화·KAI, 경남 기업들이 우주 발사 선도",https://news.kbs.co.kr/news/view.do?ncd=530752...,[앵커] 순수 국내 기술로 제작된 우주 발사체 '누리호' 개발 과정에는 300여...
8,김 총리 “성과 내기 쉬운 연구에만 예산 투입했는지 반성해야”,https://news.kbs.co.kr/news/view.do?ncd=535428...,김부겸 국무총리는 “관성에 빠져 정말 필요한 과제가 아닌 성과를 내기 쉬운 연구에만...
9,러 항공우주 전시회…첨단 기술 한자리에,http://news.kbs.co.kr/news/view.do?ncd=3519196...,<앵커 멘트> 2년마다 열리는 러시아 최대 규모의 항공우주 전시회가 개최됐습니다. ...
10,미사일 지침 종료…경남 방산·우주항공 ‘기대’,https://news.kbs.co.kr/news/view.do?ncd=519208...,한미 정상회담에서 미사일 지침이 종료됨에 따라 경남의 방산과 우주항공업계의 기대감이...


## RISS

정보기술만 함

In [166]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from pandas import Series, DataFrame
import time
import urllib

In [167]:
Delay=2 #time.sleep(Delay)로 일괄 적용

In [168]:
def Preceding_work(searchQ):#셋팅
    global driver
    global xpath
    
    # path = './chromedriver.exe'
    path = 'C:/chromedriver/chromedriver.exe'
    driver = webdriver.Chrome(path)
    xpath = driver.find_element_by_xpath

    #검색
    url='http://www.riss.kr/search/Search.do?colName=all&isDetailSearch=N&searchGubun=true&oldQuery=&sflag=1&fsearchMethod=search&isFDetailSearch=N&searchQuery={}&kbid=&pageNumber=1&query={}'.format(searchQ, searchQ)
    driver.get(url)
    time.sleep(Delay)
    essay_click_btn = xpath('//*[@class="tabM1 "]/a')
    driver.execute_script("arguments[0].click();",essay_click_btn)
    time.sleep(Delay)

    #100개씩 보기 설정
    click_btn = xpath('//*[@class="selectBoxW"]')
    driver.execute_script("arguments[0].click();",click_btn)
    time.sleep(Delay)
    click_btn = xpath('//*[@class="optList"]/ul/li[4]/a')
    driver.execute_script("arguments[0].click();",click_btn)
    time.sleep(5)
    click_btn = xpath('//*[@class="listSearch"]')
    driver.execute_script("arguments[0].click();",click_btn)
    time.sleep(Delay)

In [169]:
def riss_crawling(searchQ, number):#검색어, 크롤링 갯수
    Preceding_work(searchQ)#사전 셋팅 함수
    
    page=1#페이지 수
    url=[]#url
    title=[]#제목
    year=[]#날짜
    cur_number=0#현재 크롤링수
    contents=[]
    
    #url 및 제목 날짜 수집
    try:
        while True:
            if cur_number==number:#현재 크롤링 수가 목표 치라면:
                break
            
            # titles = driver.find_elements_by_css_selector(".title")#제목,url
            titles = driver.find_elements_by_xpath('//*[@class="srchResultListW"]/ul/li/div/p/a')
            years=driver.find_elements_by_css_selector(".assigned")#날짜

            for i in range(len(titles)):
                if cur_number==number:#현재 크롤링 수가 목표 치라면:
                    break
                
                cur_number+=1#현재 크롤링수
                #url 경로 따라가기
                # href=titles[i].find_element_by_tag_name('a')
                # href=href.get_attribute('href')
                href=urllib.parse.unquote(titles[i].get_attribute('href'))
                
                title.append(titles[i].text)#제목 수집
                url.append(href)#url수집
                year.append(years[i].text)#발행일 수집
                
            else:#한페이지 모두 크롤링 했을 시
                print(page,len(url))
                if page<10:#1~10페이지
                    page+=1
                    serach_click_btn = xpath("//*[@id='pcPaging{0}']".format(page))
                    driver.execute_script("arguments[0].click();",serach_click_btn)
                    time.sleep(Delay)
                elif page%10==0:#10페이지째, 20페이째....
                    page+=1
                    serach_click_btn = xpath("//*[@id='next']/img")
                    driver.execute_script("arguments[0].click();",serach_click_btn)
                    time.sleep(Delay)
                else:#나머지
                    page+=1
                    serach_click_btn = xpath("//*[@id='pcPaging{0}']".format(page))
                    driver.execute_script("arguments[0].click();",serach_click_btn)
                    time.sleep(Delay)
        
    
        #url 따라서 초록 수집
        for i in url:
            driver.get(i)
            time.sleep(Delay)
            # content=driver.find_elements_by_css_selector(".abstractWrap.eHideSec1.detail-con1")
            try : 
                content = driver.find_elements_by_xpath('//*[@class="content additionalInfo"]/div/div[2]/p')
                content_text=content[0].text
                contents.append(content_text)
            except : 
                contents.append('X')
                # contents.append(content[0].text)
                #for j in content:
                #    contents.append(j.text)
    except:#에러시 현재하끼 한거 반환
        return title,url,year,contents
    
    driver.close()
    
    return title,url,year,contents

In [171]:
#main
title,url,year,contents=riss_crawling("정보기술", 5)

#데이터 프레임 만들기
raw_data = {'title': title,
            'url': url,
            'year': year,
            'content':contents}

df=DataFrame(raw_data)
df.to_csv("정보기술.csv",index=False, encoding="utf-8-sig")#csv로 저장

C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/471829790.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/471829790.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  essay_click_btn = xpath('//*[@class="tabM1 "]/a')
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/471829790.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  click_btn = xpath('//*[@class="selectBoxW"]')
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/471829790.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  click_btn = xpath('//*[@class="optList"]/ul/li[4]/a')
C:\Users\jessy\AppData\Local\Temp/ipykernel_17664/471829790.py:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead